# Motivation
论文原文：[《Neural Factorization Machines for Sparse Predictive Analytics》](https://arxiv.org/pdf/1708.05027)

参考学习地址：[推荐系统系列（七）：NFM理论与实践](https://zhuanlan.zhihu.com/p/92293407)

NFM（Neural Factorization Machine）同样在FM基础上引入DNN，利用非线性结构来学习更多数据信息，但创新提出使用Bi-interaction Layer（Bi-linear interaction）结构来对二阶交叉信息进行处理，为非例如Wide&Deep对二阶交叉特征向量仅进行简单concatenation后送入DNN。

$$
\begin{align}
\hat{y}_{NFM}(X)=w_0+\Sigma_{i=1}^nw_ix_i+f(X)
\end{align}
$$

下图中未涵盖一阶项与偏置，聚焦于$f(X)$部分

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/nfm.png" width = "600" height = "500" alt="NFM" align=center />

 ## Embedding Layer
 Embedding vector同样通过lookup table获取，最终输入特征向量是由输入特征值$x_i$与embedding vector $v_i$相乘得到，i.e. $V_x=\{x_1v_1,...,x_nv_n\}$。与大部分模型相同

## Bi-interaction Layer
作为NFM核心，本质是一个pooling操作，将embedding vector集合归并为一个向量
$$
\begin{align}
f_{BI}(V_x)=\Sigma_{i=1}^n\Sigma_{j=i+1}^nx_iv_i\odot x_jv_j
\end{align}
$$

其中 $\odot$ 代表哈达玛积（见wide$deep部分，i.e. 逐元素相乘结果为向量）。所以Bi-interation Layer将embedding vectors进行两两交叉后，逐元素相乘再对应位置求和pooling，最终 $f_{BI}(V_x)$ 为一个同embeddingg维的向量。

对比FM的二阶交互项，假设有稀疏输入向量$x=(x_1,...,x_n)$，每个特征$i$有一个对应隐向量$v_i\in \R^k$，其中$k$为嵌入维度。FM的二阶交互项：
$$
\begin{align}
f_{FM}(V_x)=\Sigma_{i=1}^n\Sigma_{j=i+1}^n<v_i,v_j>x_ix_j
\end{align}
$$

FM思路是计算所有非零特征对隐向量内积的加权和，得到的是一个标量。

假设输入$x=(x_1,x_2,x_3)$，嵌入维度维度$k=2$，则有$v^{(1)}=(v_1^{(1)},v_2^{(1)}), v^{(2)}=(v_1^{(2)},v_2^{(2)}),v^{(3)}=(v_1^{(3)},v_2^{(3)})$。

FM计算如下：
* 先计算每对特征内积
$$
\begin{align}
<v^{(1)},v^{(2)}>&=v_1^{(1)}v_1^{(2)}+v_2^{(1)}v_2^{(2)}\\
<v^{(1)},v^{(3)}>&=v_1^{(1)}v_1^{(3)}+v_2^{(1)}v_2^{(3)}\\
<v^{(2)},v^{(3)}>&=v_1^{(2)}v_1^{(3)}+v_2^{(2)}v_2^{(3)}\\
\end{align}
$$
* 然后求和
$$
\begin{align}
f_{FM}(V_x)=(v_1^{(1)}v_1^{(2)}+v_2^{(1)}v_2^{(2)})x_1x_2+(v_1^{(1)}v_1^{(3)}+v_2^{(1)}v_2^{(3)})x_1x_3+(v_1^{(2)}v_1^{(3)}+v_2^{(2)}v_2^{(3)})x_2x_3
\end{align}
$$


BI计算如下：
* 先计算哈达玛积
$$
\begin{align}
v^{(1)}\odot v^{(2)}&=(v_1^{(1)}v_1^{(2)},v_2^{(1)}v_2^{(2)})\\
v^{(1)}\odot v^{(3)}&=(v_1^{(1)}v_1^{(3)},v_2^{(1)}v_2^{(3)})\\
v^{(2)}\odot v^{(3)}&=(v_1^{(2)}v_1^{(3)},v_2^{(2)}v_2^{(3)})\\
\end{align}
$$
* 求和池化
$$
\begin{align}
f_{BI}(V_x)&=(v_1^{(1)}v_1^{(2)}x_1x_2,v_2^{(1)}v_2^{(2)}x_1x_2)+(v_1^{(1)}v_1^{(3)}x_1x_3,v_2^{(1)}v_2^{(3)}x_1x_3)+(v_1^{(2)}v_1^{(3)}x_2x_3,v_2^{(2)}v_2^{(3)}x_2x_3)\\
&=(v_1^{(1)}v_1^{(2)}x_1x_2 + v_1^{(1)}v_1^{(3)}x_1x_3 + v_1^{(2)}v_1^{(3)}x_2x_3, v_2^{(1)}v_2^{(2)}x_1x_2 + v_2^{(1)}v_2^{(3)}x_1x_3 + v_2^{(2)}v_2^{(3)}x_2x_3)
\end{align}
$$

上式可以得到结论：<font color="red">FM是BI的元素和，i.e. $ f_{FM}=\Sigma f_{BI} $</font>

为了降低时间复杂度$O(kn^2)\rightarrow O(kN_x)$，其中$N_x$维输入特征$X$的非零元素个数。同FM相同的改写优化为：
$$
\begin{align}
f_{BI}(V_x)=\frac{1}{2}[(\Sigma_{i=1}^{n}x_iv_i)^2-\Sigma_{i=1}^n(x_iv_i)^2]
\end{align}
$$

Bi-interaction Layer与FM中的二阶交叉项对比，没有引入额外的参数，同时也能以线性时间复杂度训练。

## NN结构
NN部分就是传统的MLP。但注意，如果加上一阶项和偏置项，NFM与Wide&Deep极为相似，但不同的是
* Wide&Deep的NN部分输入为Sparse一阶特征的向量表示，人工的交叉部分实则放在了Wide部分和一阶共同作为输入
* NFM的NN部分使用的是Bi-interaction交叉后的结果，Wide部分使用的是一阶项和偏置。

所以<font color="red"> **Wide&Deep相当于一阶和高阶交叉并联，而NFM是串联结构。**</font>

相比于稍早的DeepFM，FM部分使用内积计算二阶交互，NFM的Bi-Interaction层使用哈达玛积+池化的方式，将二阶交互的计算结果从标量扩展为向量，这个思路在后续的xDeepFM的CIN中也有体现。在DNN的输入中，DeepFM使用所有特征的嵌入向量直接concat（维度为m × k（m=特征数，k=嵌入维度）），而NFM使用Bi-interaction层的池化输出向量（固定维度k）

## 模型训练
模型复杂度提高后，会导致过拟合，NFM中使用了dropout和batch normalization的方法缓解。同时要注意先后顺序，因为需要为激活函数提供需要的数据分布，所以应该在Bi-Interaction Layer之后接入batch normalization，然后直接进行dropout。需要注意的是，Bi-Interaction Layer是没有激活函数的。后续隐层需要进行batch normalization调整数据分布，然后再加上激活函数，最后使用dropout技术。

# 一些思考
由于FM是BI（两两交叉）的元素和，而BI后的输入又要喂入MLP中本质又是加权和，所以相当于FM认为所有隐维度权重都是1（贡献相等），而NFM（没有NN层时）每个隐维度有不同的权重。所以NFM的本质是：先用BI层将FM的标量交互扩展为向量交互，再用DNN学习这些交互向量的复杂组合模式。这比简单的加权求和强大得多